In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import sys
sys.path.append("/content/drive/MyDrive/Colab Notebooks/PML-MD")

In [29]:
import torch
import time
import os
import torch.optim as optim
import numpy as np
from model import LinearNet
from dataset import get_loader
from scipy.io import savemat
from criteria import hLoss, rLoss, oError, Conv, avgPre
from torch.utils.data import TensorDataset, DataLoader

In [30]:
def get_loader(dataname,batch_size,cv_num,p_noise,p_true,meta_size=50):
  #環境によって変える。kind_of_dataのディレクトリ
  data_path = f"/content/drive/MyDrive/Colab Notebooks/new_data2/" + dataname + "/"
  features = np.loadtxt(data_path+"data.csv", delimiter=',')
  labels = np.loadtxt(data_path+"target.csv", delimiter=',',dtype = float)
  if dataname in ['mirflickr', 'music_emotion', 'music_style','YeastBP']:
    plabels = np.loadtxt(data_path+"cand/0.csv", delimiter=',',dtype = float)
  else:
    plabels = np.loadtxt(data_path+"cand/"+str(p_noise)+".csv", delimiter=',',dtype = float)
  cv_inds = np.loadtxt(data_path+"index/5-cv.csv",delimiter=',',dtype = int)-1

  features_num = features.shape[1]
  labels_num = labels.shape[1]

  #全体の特徴をtensorに変換
  features = torch.tensor(features, dtype=torch.float)
  labels = torch.tensor(labels, dtype=torch.float)
  plabels = torch.tensor(plabels, dtype=torch.float)

  zeros = torch.zeros_like(labels)
  labels = torch.where(labels == -1, zeros, labels)
  plabels = torch.where(plabels == -1, zeros, plabels)

  #訓練とテストに分割(自分は5分割交差検証)
  # split training into train and test set
  """n = len(features)
  train_size = int(n * prec)"""
  
  #trainとtestのインデックス(自分はcv_inds)
  """indices = torch.randperm(n)
  train_idxs = indices[:train_size]
  test_idxs = indices[train_size:]"""
  index = np.where(cv_inds!=cv_num)
  index = np.ravel(index[0])
  train_idxs = torch.tensor(index)
  index = np.where(cv_inds==cv_num)
  index = np.ravel(index[0])
  test_idxs = torch.tensor(index)

  #上で指定したインデックスでtrainとtestに分ける             
  train_features = torch.index_select(features, 0, train_idxs)
  train_labels = torch.index_select(labels, 0, train_idxs)
  plabels = torch.index_select(plabels, 0, train_idxs)

  clean_features = train_features.clone()
  clean_labels = train_labels.clone()

  noisy_features = train_features.clone()
  noisy_labels = plabels.clone()

  test_features = torch.index_select(features, 0, test_idxs)
  test_labels = torch.index_select(labels, 0, test_idxs)

  #検証集合を作る
  # select a clean batch from training set
  n = len(train_features)
  # meta_size = batch_size
  indices = torch.randperm(n)
  meta_idxs = indices[:meta_size]
  train_idxs = indices[meta_size:]

  meta_features = torch.index_select(train_features, 0, meta_idxs)
  meta_labels = torch.index_select(train_labels, 0, meta_idxs)

  train_features = torch.index_select(train_features, 0, train_idxs)
  train_labels = torch.index_select(plabels, 0, train_idxs)

  train_dataset = TensorDataset(train_features, train_labels)
  test_dataset = TensorDataset(test_features, test_labels)
  meta_dataset = TensorDataset(meta_features, meta_labels)
  clean_dataset = TensorDataset(clean_features, clean_labels)
  noisy_dataset = TensorDataset(noisy_features, noisy_labels)

  train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
  test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)
  meta_loader = DataLoader(dataset=meta_dataset, batch_size=batch_size, shuffle=True)
  clean_loader = DataLoader(dataset=clean_dataset, batch_size=batch_size, shuffle=True)
  noisy_loader = DataLoader(dataset=noisy_dataset, batch_size=batch_size, shuffle=True)

  return train_loader, test_loader, meta_loader, clean_loader, noisy_loader, features_num, labels_num

In [31]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')
data_list = ["emotions","enron","CAL500","scene","genbase","mirflickr","music_style","music_emotion","YeastBP"]
batch_size_list = [50,100,50,200,50,500,200,200,200]

kind = 0
dataname = data_list[kind]
batch_size = batch_size_list[kind]
cv_num = 0
p_noise = 50
p_true = 1

In [32]:
train_loader, test_loader, meta_loader, clean_loader, noisy_loader, features_num, labels_num = get_loader(dataname,batch_size,cv_num,p_noise,p_true)

In [33]:
print(test_loader)